# IC12 NetCDF Production

The following script is used to produce a NetCDF4 file containing the data on the _IC12_ sheet of the _Datasets_IC12_FK17_TIR18_SarahWauthy.xlsx_ file.

In [1]:
import xarray as xr
import numpy as np
from netCDF4 import Dataset
import pandas as pd
import os
import bisect
import matplotlib.pyplot as plt

Files are pulled and saved from the local working directory so we use strings with directory filepaths and the os module to switch between directories for loading and saving.

In [2]:
rawfiledirectory = r"C:\Users\Alfie\Desktop\IMAS\ncfilesraw"
outfiledirectory = r"C:\Users\Alfie\Desktop\IMAS\ncfiles"
os.chdir(rawfiledirectory)
filename = 'Datasets_IC12_FK17_TIR18_SarahWauthy.xlsx'
sheet = 'IC12'

The data is read in and empty rows are removed.

In [3]:
chemistry = pd.read_excel(filename, sheet_name = sheet, 
                          header=3, usecols = "E:J").to_numpy()
chemistry = chemistry[~np.isnan(chemistry).all(axis=1), :]

Manual delete of row 773 as there seems to be an incorrect record at 780 in xlsx and it messes up the operations ahead if not removed.

In [4]:
if(sheet == 'IC12'):
    chemistry = np.delete(chemistry,773,axis=0)

Get the agemodel depths and years into separate arrays (easier for bisect)

In [5]:
agedepths = pd.read_excel(filename, sheet_name = sheet, 
                          header=3, usecols = "A", nrows=270).squeeze()
ageyears = pd.read_excel(filename, sheet_name = sheet, 
                         header=3, usecols = "B", nrows=270).squeeze()

Stack a zeros column (index 6) alongside the chemistry data for a year dimension

In [6]:
z = np.zeros((len(chemistry),1), dtype=int)
chemistry = np.append(chemistry, z, axis=1)

Records which cross a depth-year boundary in the age model are split into two records with the boundary depth being used as the top/bottom depth in the two respective splits, with all chemistry data being copied into both. Bisect returns the index for this depth in agedepths, giving the following index if equal to a depth-year boundary.

In [ ]:
for i in range(len(chemistry)):
    # Get agemodel indices for top and bottom depths
    if(not(pd.isna(chemistry[i,0]))):
        t = bisect.bisect(agedepths, chemistry[i,0]) 
        b = bisect.bisect(agedepths, chemistry[i,1])
        chemistry[i,6] = ageyears[t]
        if(t != b):
            split = chemistry[i] #duplicate the record that crosses a boundary
            #set the top of the split sample to be the boundary, append
            split[0] = agedepths[bisect.bisect(agedepths, chemistry[i,0])]
            split[6] = ageyears[b] #Set new year
            chemistry = np.vstack([chemistry, split])
            
            #set bottom of original sample to be the boundary
            chemistry[i,1] = agedepths[bisect.bisect(agedepths,
                                                     chemistry[i,0])] 

Then sort in ascending order of topdepth.

In [ ]:
chemistry = chemistry[chemistry[:, 0].argsort()]

Next we change to the save directory and create a netCDF file.

In [ ]:
os.chdir(outfiledirectory)
ncout = Dataset('IC12.nc','w','NETCDF4') # using netCDF3 for output format 
ncout.description = "IC12 ice core chemical records, published in Philippe et al., 2016. IC12 position: -70.234690S, 26.334900E"

Create the topdepth dimension and all of the chemistry variables.

In [ ]:
ncout.createDimension('topdepth',size=None)
yearvar = ncout.createVariable('year','int',('topdepth'))
yearvar[:] = chemistry[:,6]
tdvar = ncout.createVariable('topdepth','float32',('topdepth'))
tdvar[:] = chemistry[:,0]
thickvar = ncout.createVariable('thickness', 'float32',('topdepth'),fill_value=np.nan)
thickvar.description = "Thickness of this sample"
navar = ncout.createVariable('Na', 'float32',('topdepth'),fill_value=np.nan)
navar.units = "ppb"
msavar = ncout.createVariable('MSA', 'float32',('topdepth'),fill_value=np.nan)
msavar.units = "ppb"
so4var = ncout.createVariable('SO4', 'float32',('topdepth'),fill_value=np.nan)
so4var.units = "ppb"

Insert all the data.

In [ ]:
for i in range(len(chemistry)):
    thickvar[i] = chemistry[i,1]-chemistry[i,0]
    navar[i] = chemistry[i,3]    
    msavar[i] = chemistry[i,4]
    so4var[i] = chemistry[i,5]

In [ ]:
ncout.close()